In [3]:
import pandas as pd
import os

# Find counties

In [61]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,county,state
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,False,True,False,Day,Day,Day,Day,East Baton Rouge,Louisiana
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,False,Night,Night,Night,Night,Loudoun,Virginia
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,True,False,Day,Day,Day,Day,Santa Barbara,California
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,False,Day,Day,Day,Day,Mower,Minnesota
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,Night,Night,Night,Night,Kern,California


In [ ]:

# Load the U.S. counties shapefile (replace the path with the correct path to the shapefile)
counties_shapefile = "../shapefiles/cb_2021_us_county_20m.shp"  # Example path
gdf_counties = gpd.read_file(counties_shapefile)

# Ensure the CRS (coordinate reference system) is the same for the data
gdf_counties = gdf_counties.to_crs(epsg=4326)  # Convert to WGS84 if needed

# Read the dataset with accident data
us_accidents_df = pd.read_csv("../US_Accidents_small.csv")


In [ ]:

# Function to get county and state using geopandas and a shapefile
def get_county_state(lat, lng):
    point = Point(lng, lat)  # Create a Point object from the longitude and latitude
    for _, county_row in gdf_counties.iterrows():
        if county_row['geometry'].contains(point):  # Check if the point is inside the county
            county = county_row['NAME']
            state = county_row['STATE_NAME']
            return county, state
    return None, None  # If no match is found


In [ ]:
# Apply the function to get county and state
us_accidents_df['county'], us_accidents_df['state'] = zip(*us_accidents_df.apply(
    lambda row: get_county_state(row['Start_Lat'], row['Start_Lng']), axis=1
))


In [64]:

# Display the results
us_accidents_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,county,state
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,False,True,False,Day,Day,Day,Day,East Baton Rouge,Louisiana
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,False,Night,Night,Night,Night,Loudoun,Virginia
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,True,False,Day,Day,Day,Day,Santa Barbara,California
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,False,Day,Day,Day,Day,Mower,Minnesota
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,Night,Night,Night,Night,Kern,California


In [65]:
# Drop rows where county or state is None
us_accidents_df_processed = us_accidents_df.dropna(subset=['county', 'state'])

us_accidents_df_processed.head()


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,county,state
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,False,True,False,Day,Day,Day,Day,East Baton Rouge,Louisiana
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,False,Night,Night,Night,Night,Loudoun,Virginia
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,True,False,Day,Day,Day,Day,Santa Barbara,California
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,False,Day,Day,Day,Day,Mower,Minnesota
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,Night,Night,Night,Night,Kern,California


In [67]:
us_accidents_df_processed.county.value_counts()

county
Los Angeles    179
Orange          94
Miami-Dade      84
Harris          56
Mecklenburg     53
              ... 
Millard          1
Darlington       1
Winchester       1
White            1
Herkimer         1
Name: count, Length: 437, dtype: int64

In [68]:
us_accidents_df_processed.to_csv("../US_Accidents_small_processed.csv", index=False)

# Add months


In [72]:
# Read the dataset with accident data
us_accidents_df = pd.read_csv("../US_Accidents_small.csv")
# Datetime functions
us_accidents_df_processed['Start_Time'] = us_accidents_df_processed['Start_Time'].str.split('.').str[0]  # Remove milliseconds
us_accidents_df_processed['Month'] = pd.to_datetime(us_accidents_df_processed['Start_Time']).dt.strftime('%b')
us_accidents_df_processed.to_csv("../US_Accidents_small_processed.csv", index=False)

In [73]:
us_accidents_df_processed.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,county,state,Month
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,True,False,Day,Day,Day,Day,East Baton Rouge,Louisiana,June
1,A-4694324,Source1,2,2022-12-03 23:37:14,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,Night,Night,Night,Night,Loudoun,Virginia,December
2,A-5006183,Source1,2,2022-08-20 13:13:00,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,True,False,Day,Day,Day,Day,Santa Barbara,California,August
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,Day,Day,Day,Day,Mower,Minnesota,February
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,Night,Night,Night,Night,Kern,California,December
